# Pandas Tricks

*Rappel et/ou découverte de manipulation de matrices de données avec pandas*

### Lire un tableur



In [1]:
import pandas as pd

# préciser le chemin du fichier à lire
cheminDuFichierCSV = "./../data/dictionary/1757_Buchanan-J.csv" 

# demander à pandas de lire le fichier et garder ce tableau accessible dans une variable
dfDicoBuch = pd.read_csv(cheminDuFichierCSV, delimiter=";", encoding="utf8")

# afficher à l'écran un échantillon du tableau
dfDicoBuch.sample(n=5)

,nbSyll,pron,origin,warning,remarques,POS,lineId,pageId
12730,3,ſoʹlĕmnīze,L.,NaN,NaN,v.,53.0,198
12202,1,ſcrănch,NaN,NaN,NaN,v.,29.0,191
742,3,ăʹrrŏgănce,L.,NaN,NaN,n.,60.0,29
12648,1,ſmooth,NaN,NaN,NaN,a.,43.0,197
12147,2,ſcēʹn_e_ry,NaN,NaN,NaN,n.,46.0,190


Pour lires des tableurs aux format xls, xlsx ou odt il faut utiliser pd.read_excel(cheminDuFichier) et je conseil de bien préciser explicitement l'engine à utiliser en paramètre.
* dfXlsx = pd.read_excel(cheminDuFichierXLSX, engine="openpyxl")
* dfOds = pd.read_excel(cheminDuFichierODS, engine="odf")

### Ajouter un nouvelle colonne contenant le resultat d'un traitement

In [2]:
# définir une fonction qui retourne quelque chose
def sizePron(strPron):
    #retourner le nombre de caractère de la prononciation
    
    return len(strPron)

In [3]:
# appeler cette fonction sur toutes les lignes de la matrice pour remplir une nouvelle colonne grace à apply
dfDicoBuch['sizePron'] = dfDicoBuch.apply(lambda x: sizePron(x['pron']),1)

dfDicoBuch.sample(n=5)

,nbSyll,pron,origin,warning,remarques,POS,lineId,pageId,sizePron
8653,2,mĭʹſsĭve,L.,NaN,NaN,a.,58.0,141,8
15117,2,ŭnſcrew,NaN,nbSyll>1 et pas d'accent tonic,NaN,v.,71.0,231,7
14635,2,ŭʹmb_e_r,NaN,NaN,NaN,n.,21.0,225,8
15247,3,ŭnyieʹlding,NaN,NaN,NaN,a.,57.0,233,11
14348,2,vaʹlĕnce,NaN,NaN,NaN,n.,19.0,221,8


## Un petit calcul

In [4]:
dfDicoBuch['sizePron'].median()

9.0

## Une recherche de valeur

In [5]:
dfDicoBuch[dfDicoBuch['sizePron']==26.0]

,nbSyll,pron,origin,warning,remarques,POS,lineId,pageId,sizePron
2813,3,contrŏʹller or comptrollĕr,NaN,NaN,NaN,n.,49.0,58,26
7492,3,kaʹlĕnd_a_r or caʹlĕnd_a_r,NaN,NaN,NaN,n.,2.0,125,26


## Afficher des graphiques
Plusieurs librairie possibles :
* matplotlib (static, bonne qualité pour publication)
* seaborn (interactif, pour le web)
* bokeh (interactif, pour le web)
* plotly (interactif, pour le web)

In [6]:
import seaborn as sns
import matplotlib.pyplot as plt

In [7]:
%matplotlib ipympl

plt.figure("scatter plot view")
sns.scatterplot(data=dfDicoBuch, x="nbSyll", y="sizePron", hue="POS")

ModuleNotFoundError: No module named 'ipympl'

In [ ]:
%matplotlib ipympl

plt.figure("scatter plot view")
sns.scatterplot(data=dfDicoBuch, x="nbSyll", y="sizePron", hue="POS", alpha=0.05)

## Manipuler des matrices de format exotique

In [8]:
cheminDuDicoBigi = "./../data/dictionary/2015_Bigi-B.dict" 

# ouvrir une matric exotique
#dfDicoBigi = pd.read_csv(cheminDuDicoBigi, delimiter=" [] ", encoding="utf8")

dfDicoBigi = pd.read_csv(cheminDuDicoBigi, delimiter=" \[\] ", encoding="utf8", engine='python', names=['hwd','Sampa'], skiprows=8 )

# afficher à l'écran un échantillon du tableau
dfDicoBigi.head(n=5)

,hwd,Sampa
0,a,@
1,a(2),eI
2,a'ready,@ r\ E d i:
3,a's,eI z
4,a.,eI


Intro/rappel Regex : https://gitlab.huma-num.fr/mnauge/cnam-athon-transcrire/-/blob/master/codes/regexBasics.ipynb

## Jouer avec deux matrices à la fois

In [9]:
cheminDuMappingSampaIpa = "./../data/dictionary/sampa-ipa.txt" 
dfMappingSampaIpa = pd.read_csv(cheminDuMappingSampaIpa, delimiter="  *", encoding="utf8", engine='python', names=['sampa','ipa'])
dfMappingSampaIpa.sample(n=3)

,sampa,ipa
5,-\,‿
18,:,ː
152,p\,ɸ


In [10]:
dfMappingSampaIpa['sampa'].value_counts()

r      3
h      2
g      2
U~/    2
@      2
      ..
O:     1
P      1
Q      1
R      1
~      1
Name: sampa, Length: 177, dtype: int64

In [11]:
dfMappingSampaIpa[dfMappingSampaIpa['sampa']=='@']

,sampa,ipa
28,@,ə
29,@,ɛ


In [12]:
# ne garder que la première occurence pour éviter une indécision mais ce n'est pas terrible. A discuter avec les linguistes.
dfMappingSampaIpa.drop_duplicates(subset=['sampa'], keep='first', inplace=True)

In [13]:
dfMappingSampaIpa[dfMappingSampaIpa['sampa']=='@']

,sampa,ipa
28,@,ə


In [14]:
def sampa2ipa(strSampa, dfMapping):
    ipaRes = ""
    
    listValSampa = strSampa.split(" ")
    #print(listValSampa)
    for valSampa in listValSampa:
        dfRowFinded = dfMapping[dfMapping['sampa']==valSampa]

        if len(dfRowFinded==1):
            ipaRes+=dfRowFinded.iloc[0]['ipa']
        else:
            ipaRes+='?'
    return ipaRes

In [15]:
sampa2ipa("A r\ t i",dfMappingSampaIpa)

'ɑɹti'

In [17]:
dfDicoBigiRich = dfDicoBigi.copy()

dfDicoBigiRich['ipa'] = dfDicoBigiRich.apply(lambda x: sampa2ipa(x['Sampa'], dfMappingSampaIpa),1)


In [18]:
dfDicoBigiRich.sample(n=5)

,hwd,Sampa,ipa
21430,clouatre,k l aU @ t r\,kl?ətɹ
14583,brody,b r\ @U 4 i:,bɹoʊɾiː
92771,promiscuous(2),p r\ A m I s k j u @ s,pɹɑmɪskjuəs
15644,burchfield,b 3:r tS f i: l d,bɜ:r?fiːld
7381,baited,b eI 4 i d,b?ɾid


### Sauvegarder la nouvelle matrice


In [19]:
pathDicoBigiRich = r"C:\Users\kandasp\Downloads\cnam-athon-pronunciation-main (1)\cnam-athon-pronunciation-main\data\dataframe\BigiRichSample.csv"
dfDicoBigiRich.to_csv(pathDicoBigiRich, sep=";",encoding="utf8",index=False)